In [1]:
import graphlab as gl
import belief_propagation as bp

[WARNING] Unable to write current GraphLab Create license to /home/yigit/.graphlab/config. Ensure that this user account has write permission to /home/yigit/.graphlab/config to save the license for offline use.
[INFO] This non-commercial license of GraphLab Create is assigned to shhong@cs.umd.edu and will expire on May 21, 2017. For commercial licensing options, visit https://turi.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-23566 - Server binary: /home/yigit/anaconda2/envs/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1468509236.log
[INFO] GraphLab Server Version: 1.6.1
[WARNING] Unable to create session in specified location: '/home/yigit/.graphlab/artifacts'. Using: '/var/tmp/graphlab-yigit/23566/tmp_session_d0bd0938-6416-4e20-9794-85cee5c5f68a'
A newer version of GraphLab Create (v2.0.1) is available! Your current version is v1.6.1.

You can use pip to upgrade the graphlab-create package. For more information see http

In [2]:
def generate_edge_potential_matrix(e):
    return [[0.5+e, 0.5-e], [0.5-e, 0.5+e]]

def generate_single_graph(nos, snp, ep_e, rnp):
    print 'Generating a graph with following parameters:'
    print 'Number of source nodes: {}'.format(nos)
    print 'Source node potentials: {}'.format(snp)
    print 'Edge potential epsilon value: {}'.format(ep_e)
    print 'Relation node potentials: {}'.format(rnp)
    
    relation_vertex = gl.Vertex(0, attr={'type':'relation', 'prior': rnp})
    source_vertices = [gl.Vertex(i, attr={'type': 'source', 'prior': snp}) for i in xrange(1,nos + 1)]
    edges = [gl.Edge(i,0, attr={'potential':generate_edge_potential_matrix(ep_e)}) for i in xrange(1, nos + 1)]
    return gl.SGraph().add_vertices([relation_vertex] + source_vertices).add_edges(edges)

# a generator yields the graphs with different parameters that is given as lists (all of the inputs are lists)
def generate_source_rel_graphs(no_of_sources, source_nps, ep_e_values, rel_nps):
    for rnp in rel_nps:
        for snp in source_nps:
            for ep in ep_e_values:
                for nos  in no_of_sources:
                    yield generate_single_graph(nos, snp, ep, rnp)

In [3]:
# parameters for simulations
no_of_sources = [1,10,100,1000] # 1 is for the single source->single relation case
source_nps = [[0.5  , 0.5], [0.9, 0.1], [0.7, 0.3]]  # node potentials values for source nodes
ep_e_values = [0.01, 0.05, 0.1] # edge potentials values
rel_nps = [[0.01, 0.99]] # node potentials for relation node

In [4]:
sim_iterator = generate_source_rel_graphs(no_of_sources, source_nps, ep_e_values, rel_nps)

for graph in sim_iterator:
    marginals = bp.belief_propagation(graph,
                                      prior='prior',
                                      edge_potential='potential',
                                      num_iterations=10,
                                      method='sum',
                                      convergence_threshold=0.01)
    
    after_bp_graph = gl.SGraph().add_vertices(graph.get_vertices().join(marginals, on='__id'))
    print after_bp_graph.get_vertices(ids=[0,1])

Generating a graph with following parameters:
Number of source nodes: 1
Source node potentials: [0.5, 0.5]
Edge potential epsilon value: 0.01
Relation node potentials: [0.01, 0.99]
PROGRESS: Validating input...
PROGRESS: Validation time: 1.18542
PROGRESS: Initializing...
PROGRESS: Initializing time: 0.994367
PROGRESS: Iteration 1:
PROGRESS: 	Sum of l2 change in messages: 0.0384464
PROGRESS: 	Iteration time in seconds: 0.809934
PROGRESS: Iteration 2:
PROGRESS: 	Sum of l2 change in messages: 0
PROGRESS: 	Iteration time in seconds: 0.731513
+------+--------------+----------+------------------+
| __id |    prior     |   type   |     marginal     |
+------+--------------+----------+------------------+
|  0   | [0.01, 0.99] | relation |   [0.01, 0.99]   |
|  1   |  [0.5, 0.5]  |  source  | [0.4902, 0.5098] |
+------+--------------+----------+------------------+
[2 rows x 4 columns]

Generating a graph with following parameters:
Number of source nodes: 10
Source node potentials: [0.5, 0.5]
Ed